# Прогнозируем задержки самолетов

In [1]:
!pip install lightgbm optuna -q

In [2]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
import pandas as pd

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [3]:
RANDOM_STATE = 111
DATASET_PATH = 'https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/flight_delays_train.csv'

In [49]:
data = pd.read_csv(DATASET_PATH)

X = data.drop('dep_delayed_15min', axis=1)
y = data['dep_delayed_15min'] == 'Y'

X.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732
1,c-4,c-20,c-3,1548,US,PIT,MCO,834
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423


Создайте список номеров колонок с категориальными признаками для бустингов

## Quiz
Какой длины получился список?

(подсказка: колонка `DepTime` числовая)

In [50]:
import numpy as np
cat_features = X.select_dtypes(include=object).columns.to_list()
cat_features

['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest']

Разобъем данные на обучение и контроль

In [63]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

In [67]:
optuna --version

NameError: name 'optuna' is not defined

## Модели с параметрами по умолчанию

Обучите CatBoost с гиперпараметрами по умолчанию.

## Quiz
Чему равен ROC-AUC на тестовых данных? Ответ округлите до сотых.

In [52]:
model = CatBoostClassifier(logging_level='Silent')
model.fit(Xtrain, ytrain,
         cat_features=cat_features,
         plot=True)
pred = model.predict_proba(Xtest)[:,1]
roc_auc = roc_auc_score(ytest, pred)
roc_auc

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0.7679447117365583

Обучите LightGBM с гиперпараметрами по умолчанию.

## Quiz
Чему равен ROC-AUC на тестовых данных? Ответ округлите до сотых.

In [64]:
# for c in X.columns:
#     col_type = X[c].dtype
#     if col_type == 'object' or col_type.name == 'category':
#         Xtrain[c] = Xtrain[c].astype('category')
#         Xtest[c] = Xtest[c].astype('category')
for cols in X.columns:
    if X[cols].dtypes == object:
        Xtrain[cols] = Xtrain[cols].astype('category')
        Xtest[cols] = Xtest[cols].astype('category')
Xtrain.dtypes

Month            category
DayofMonth       category
DayOfWeek        category
DepTime             int64
UniqueCarrier    category
Origin           category
Dest             category
Distance            int64
dtype: object

In [72]:
model_lgmn = LGBMClassifier(
)
model_lgmn.fit(Xtrain, ytrain)
pred = model_lgmn.predict_proba(Xtest)[:,1]
roc_auc_score(ytest, pred)

[LightGBM] [Info] Number of positive: 14346, number of negative: 60654
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001691 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 75000, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.191280 -> initscore=-1.441714
[LightGBM] [Info] Start training from score -1.441714


0.7341149074685321

## Optuna

Выделим дополнительную валидационную выборку.

In [66]:
Xtrain_new, Xval, ytrain_new, yval = train_test_split(Xtrain, ytrain, test_size=0.25, random_state=RANDOM_STATE)

Создайте функцию objective_lgbm, в которой среди гиперпараметров

* num_leaves = trial.suggest_int("num_leaves", 10, 100)
* n_estimators = trial.suggest_int("n_estimators", 10, 1000)

подберите оптимальные, обучая LGBM на Xtrain_new, ytrain_new и проверяя качество (ROC-AUC) на Xval.

Используйте 30 эпох обучения Optuna.


In [73]:
import optuna
from sklearn.model_selection import cross_val_score
def objective_lgbm(trial):
    num_leaves = trial.suggest_int('num_leaves', 10, 100)
    n_estimators = trial.suggest_int('n_estimators', 10, 1000)
    model = LGBMClassifier(num_leaves=num_leaves, n_estimators=n_estimators, n_jobs=-1)
    model.fit(Xtrain_new, ytrain_new)
    pred = model.predict_proba(Xval)[:,1]
    roc_auc = roc_auc_score(yval, pred)
    return roc_auc
study = optuna.create_study(direction='maximize')
study.optimize(objective_lgbm, n_trials=30)
    

[I 2024-09-06 01:23:58,354] A new study created in memory with name: no-name-1c80541c-c05e-4f68-82b5-3a52bfa41d9b


[LightGBM] [Info] Number of positive: 10730, number of negative: 45520
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001410 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 56250, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.190756 -> initscore=-1.445108
[LightGBM] [Info] Start training from score -1.445108


[I 2024-09-06 01:24:25,449] Trial 0 finished with value: 0.716458523619676 and parameters: {'num_leaves': 19, 'n_estimators': 717}. Best is trial 0 with value: 0.716458523619676.


[LightGBM] [Info] Number of positive: 10730, number of negative: 45520
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004361 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 56250, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.190756 -> initscore=-1.445108
[LightGBM] [Info] Start training from score -1.445108


[I 2024-09-06 01:24:30,150] Trial 1 finished with value: 0.723209607740176 and parameters: {'num_leaves': 27, 'n_estimators': 132}. Best is trial 1 with value: 0.723209607740176.


...

[I 2024-09-06 01:33:05,995] Trial 27 finished with value: 0.7189736839104589 and parameters: {'num_leaves': 26, 'n_estimators': 570}. Best is trial 18 with value: 0.7263426352168416.


[LightGBM] [Info] Number of positive: 10730, number of negative: 45520
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004580 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 56250, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.190756 -> initscore=-1.445108
[LightGBM] [Info] Start training from score -1.445108


[I 2024-09-06 01:33:55,249] Trial 28 finished with value: 0.7182010141555495 and parameters: {'num_leaves': 96, 'n_estimators': 414}. Best is trial 18 with value: 0.7263426352168416.


[LightGBM] [Info] Number of positive: 10730, number of negative: 45520
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005714 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 56250, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.190756 -> initscore=-1.445108
[LightGBM] [Info] Start training from score -1.445108


[I 2024-09-06 01:34:24,512] Trial 29 finished with value: 0.7160521794389004 and parameters: {'num_leaves': 21, 'n_estimators': 740}. Best is trial 18 with value: 0.7263426352168416.


In [74]:
study.best_params

{'num_leaves': 78, 'n_estimators': 28}

Обучите модель с найденными гиперпараметрами на Xtrain, ytrain и оцените ROC-AUC на тестовых данных.

In [75]:
model = LGBMClassifier(n_estimators=28, num_leaves=78)
model.fit(Xtrain, ytrain)
pred = model.predict_proba(Xtest)[:,1]
roc_auc_score(ytest, pred)

[LightGBM] [Info] Number of positive: 14346, number of negative: 60654
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001257 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 75000, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.191280 -> initscore=-1.441714
[LightGBM] [Info] Start training from score -1.441714


0.7333910096747289

## Quiz

Чему равно количество деревьев в LGBM после подбора гиперпараметров?

## Работа над улучшением модели

* Попробуйте при помощи Optuna подобрать и другие гиперпарамтеры
* Также подберите гиперпараметры у CatBoost (а не только у LightGBM)

In [ ]:
# your code here

## Quiz

Поделитесь своими результатами!